In [1]:
import git
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

import numpy as np
import pandas as pd
import json
import os

In [3]:
folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYLA\PauliDecomp\OptimizeHyperparams\Bounds")
potential = 'AHO'
cutoff = 16

dataframes = {}

folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    tolerance.append(optimzer['tolerance'])

    success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['total_VQE_time'])

    

In [4]:
df = pd.DataFrame({
    'folder': folders,
    'num_vqe': num_vqe,
    'shots': shots,
    'tolerance': tolerance,
    'Converged Runs': [f"{c}/40" for c in converged_runs],
    'maxiter': maxiter,
    'Mean Iter': mean_iters,
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [5]:
df.sort_values(['Delta min E', 'Delta median E'], ascending=[False,False])

,folder,num_vqe,shots,tolerance,Converged Runs,maxiter,Mean Iter,Delta min E,Delta median E,Exact,Time
1,Without,100,None,1.000000e-08,0/40,10000,10000,3.84427e-06,4.46272e-01,-1.16698e-03,"1 day, 10:36:59.167170"
0,With,100,None,1.000000e-08,32/40,10000,7847,1.99180e-03,4.86779e-01,-1.16698e-03,"1 day, 4:29:18.732276"


In [30]:
pd.options.display.float_format = lambda x: '{:e}'.format(x)
pd.options.display.float_format = lambda x: '{:.0e}'.format(x)

In [31]:
df['folder'] = df['folder'].astype(float)
df = df.drop(columns=['abs_tolerance','num_vqe','shots','maxiter']).sort_values('folder',ascending=False)

In [25]:
from tabulate import tabulate

In [32]:
latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex")#, floatfmt=".3f")
print(latex_table)


\begin{tabular}{rrlrlrrrrl}
\hline
   folder &   tolerance & strategy       &   popsize & Converged Runs   &   Mean Iter &   Delta min E &   Delta median E &    Exact & Time           \\
\hline
   0.1    &       0.001 & randtobest1bin &        20 & 40/40            &         105 &   0.00955877  &      0.0623874   & 0.891599 & 0:07:50.553802 \\
   0.01   &       0.001 & randtobest1bin &        20 & 40/40            &         149 &   7.31637e-14 &      4.15834e-05 & 0.891599 & 0:10:46.134155 \\
   0.001  &       0.001 & randtobest1bin &        20 & 40/40            &         210 &   7.31637e-14 &      4.13398e-05 & 0.891599 & 0:14:30.788039 \\
   0.0001 &       0.001 & randtobest1bin &        20 & 40/40            &         283 &   7.31637e-14 &      4.12992e-05 & 0.891599 & 0:19:07.060427 \\
   1e-05  &       0.001 & randtobest1bin &        20 & 40/40            &         263 &   7.31637e-14 &      4.12789e-05 & 0.891599 & 0:17:24.182418 \\
   1e-06  &       0.001 & randtobest1bin &    